In [1]:
!pip install -q langchain transformers sentence_transformers chromadb langchain_community langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.2 MB/s eta 0:00:0

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import chromadb

In [3]:
embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

embeddings = HuggingFaceEmbeddings(model_name = embedding_model_name)

<ipython-input-3-749ddfefc0ee>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
with open("recipes.txt", "r", encoding="utf-8") as file:
    text_data = file.read()

documents = [text_data]
metadatas = [{"source": "recipes.txt"}] * len(documents)

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)

text_chunks = text_splitter.create_documents(
    texts=documents,
    metadatas=metadatas
)

In [5]:
print(text_chunks[0].metadata)

{'source': 'recipes.txt'}


In [6]:
client = chromadb.Client()

vectorstore = Chroma.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    client=client,
    collection_name="recipes",
)

In [7]:
model_name = "aubmindlab/aragpt2-medium"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"  # 👈 Very important for GPT2

model = AutoModelForCausalLM.from_pretrained(model_name)

In [14]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)

llm = HuggingFacePipeline(pipeline = pipe)

Device set to use cpu


In [17]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

qa_template = """أنت مساعد طبخ ذكي.
أجب عن السؤال التالي بناءً على معرفتك والوصفة المتوفرة:

السؤال: {question}

الإجابة:"""

prompt = PromptTemplate.from_template(qa_template)

qa_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

query = "طريقة عمل الكفتة؟"
result = qa_chain.invoke({"question": query})

print("🧠 الإجابة:", result["text"])


<ipython-input-17-e55899c4d345>:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(


🧠 الإجابة: أنت مساعد طبخ ذكي.
أجب عن السؤال التالي بناءً على معرفتك والوصفة المتوفرة:

السؤال: طريقة عمل الكفتة؟

الإجابة: الطريقة : 1 - في مقلاة غير قابلة للالتصاق ضعي الزيت ثم أضيفي البصل وقلبي حتى يذبل. 2 - اضيفي اللحم واتركيه حتى ينضج. 3 - اتركي اللحم لمدة نصف ساعة إلى أن ينضج. 4 - أخرجي اللحم من المقلاة واضيفيه إلى القدر وضعيه جانبا. 5 - اغسلي اللحم جيدا بالماء البارد. 6 - تخلصي من الماء الزائد واغسليه بماء بارد. 7 - انقعي اللحم في قدر متوسط الحجم مع التقليب بين الحين والأخر للتخلص من الدهون الزائدة. 8 - رصي اللحم في طبق التقديم وزيني الطبق باللوز وبالهنا والشفا.إرسال بالبريد الإلكترونيكتابة مدونة حول هذه المشاركة المشاركة في Twitter المشاركة في Facebook المشاركة على Pinterestمن هو الرجل الذي تنجذب إليه المرأة ؟ سؤال يدور في ذهن العديد من الرجال ، لذلك قامت العديد من مواقع الانترنت باستفتاءات ودراسات تبين...كثيرة هي الاشياء التي تفرضها الحياة علينا ولا دخل لنا بها ومن ضمنها الاسماء التي ترفقنا من ايامنا الاولى في هذه الحياة والتي تبقى في حياتنا تحتوي من مواضيع مختلفة وتتشتت روتينا